In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

In [ ]:
!unzip creditcardfraud.zip

In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
df = pd.read_csv("/content/creditcard.csv")

In [ ]:
X=df.drop(columns=["Class"])
y=df["Class"]

In [ ]:
# We standarize
from sklearn import preprocessing

names=X.columns
scaled_df = preprocessing.scale(X)

scaled_df = pd.DataFrame(scaled_df,columns=names)
scaled_df

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, y,
                                                    test_size = 0.30, random_state = 0, shuffle = True, stratify = y)

In [ ]:
# Smote
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 33)
X_train_new, y_train_new = sm.fit_sample(X_train, y_train.ravel())

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization

model = Sequential()
model.add(Dense(X_train_new.shape[1], activation = 'relu', input_dim = X_train_new.shape[1]))
model.add(BatchNormalization())

model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy')

In [ ]:
history = model.fit(x=X_train_new, y=y_train_new, batch_size = 256, epochs=150,
          validation_data=(X_test, y_test))

In [ ]:
import matplotlib.pyplot as plt

evaluation_metrics=pd.DataFrame(model.history.history)
evaluation_metrics.plot(figsize=(10,5))
plt.title("Loss for both Training and Validation", size = 20)


In [ ]:
y_pred = model.predict_classes(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

cm_nn=confusion_matrix(y_test, y_pred)
cm_nn